In [1]:
"""
    处理文件，读入，配对并保存到一个数据集

"""
# 在当前目录下定义的数据集文件夹
DataDir = "Proteinfasta/"
"""
    其他的随便写点什么

"""
DataPairs=[
    ["CF_TR_Sequence.fasta.csv","CF_TE_Sequence.fasta.csv"],
    ["CRYS_TR_Sequence.fasta.csv","CRYS_TE_Sequence.fasta.csv"],
    ["MC_TR_Sequence.fasta.csv","MC_TE_Sequence.fasta.csv"],
    ["MF_TR_Sequence.fasta.csv","MF_TE_Sequence.fasta.csv"],
    ["PF_TR_Sequence.fasta.csv","PF_TE_Sequence.fasta.csv"],
    ["Combined_TrainSet.csv","Combined_TestSet.csv"]
]

In [2]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import copy
import torch.nn.functional as F


ModelDir= "Models/bi-LSTM-all_l1_e2_ba32.pth" # 在当前目录下定义的模型集文件夹
Set2Train= DataPairs[5] # 要训练的数据集
vocab_size = 100   #词表大小
embedding_size = 32   #词向量维度
num_classes = 2    #二分类
sentence_max_len = 1000  #单个句子的长度
hidden_size = 16

num_layers = 1  #一层lstm
num_directions = 2  #双向lstm
lr = 1e-3
batch_size = 8   
epochs = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
"""
这里进行数据探索

"""
def Load_Exploring(pair,pos,length:int=1000):
    df=pd.read_csv(DataDir+pair[pos])
    df["Length"]=df["Text"].apply(len)
    print("Data set is %s " % pair[pos])
    print("Max length of Text on this data set: "+ str(df["Length"].max()))
    print("Labels are: "+ str(df["y"].unique()))
    df["Length"].plot(kind="hist",bins=50,title=pair[pos]+" Length distribution")
    # for d in df.groupby(["y"]):
    #     d[1].plot(kind="hist",bins=100,title=str(d[0])+ " Length distribution")
    #     print(str(d[0])+" shape is "+ str(d[1].shape))
    print("{0}  of the proteins have a Length less than {1}".format(round(((df[df["Length"]<length].shape[0]/df.shape[0])*100),2),length))
    
# Params: {0}:dataset {1}:train or test set {2}:length of the data to explore
# Load_Exploring(DataPairs[0],0,1000)
# Load_Exploring(DataPairs[1],0,1000)
# Load_Exploring(DataPairs[2],0,1000)
# Load_Exploring(DataPairs[3],0,1000)
# Load_Exploring(DataPairs[4],0,1000)

In [4]:
"""
Here exploring the data set and trying to get balanced data for training

"""
dftr=pd.read_csv(DataDir+Set2Train[0])
print("Proportion of X,y in Train Set:")
print(dftr["y"].value_counts()/len(dftr))
print("Num of TrainSet: %s" % len(dftr))
dfte=pd.read_csv(DataDir+Set2Train[1])
print("Proportion of X,y in Test Set:")
print(dfte["y"].value_counts()/len(dfte))
print("Num of TestSet: %s" % len(dfte))
print("Prop of Train:Test is {0}:{1}".format(len(dftr)/(len(dftr)+len(dfte)),len(dfte)/(len(dftr)+len(dfte))))

Proportion of X,y in Train Set:
0    0.794629
1    0.205371
Name: y, dtype: float64
Num of TrainSet: 53245
Proportion of X,y in Test Set:
0    0.796378
1    0.203622
Name: y, dtype: float64
Num of TestSet: 13309
Prop of Train:Test is 0.8000270457072453:0.19997295429275475


In [3]:
"""
Here Define the data loader device

"""
def Character2Sent(chars):
    out=""
    for c in chars:
        out=out+c+" "
    return out[:-1]

class DataLoader(object):
    def read_raw(self,file):
        data=[]
        label=[]
        df=pd.read_csv(file)
        data=df["Text"].apply(Character2Sent)
        label=df["y"].apply(lambda x:[1,0] if x==1 else [0,1])
        return list(data),list(label)

    def word_count(self, datas):
        #统计单词出现的频次，并将其降序排列，得出出现频次最多的单词
        dic = {}
        for data in datas:
            data_list = data.split()
            for word in data_list:
                word = word.lower() #所有单词转化为小写
                if(word in dic):
                    dic[word] += 1
                else:
                    dic[word] = 1
        word_count_sorted = sorted(dic.items(), key=lambda item:item[1], reverse=True)
        return  word_count_sorted

    def word_index(self, datas, vocab_size):
        #创建词表
        word_count_sorted = self.word_count(datas)
        word2index = {}
        #词表中未出现的词
        word2index["<unk>"] = 0
        #句子添加的padding
        word2index["<pad>"] = 1
        #词表的实际大小由词的数量和限定大小决定
        vocab_size = min(len(word_count_sorted), vocab_size)
        for i in range(vocab_size):
            word = word_count_sorted[i][0]
            word2index[word] = i + 2
          
        return word2index, vocab_size

    def get_datasets(self,data_set,vocab_size, embedding_size, max_len):
        #注，由于nn.Embedding每次生成的词嵌入不固定，因此此处同时获取训练数据的词嵌入和测试数据的词嵌入
        #测试数据的词表也用训练数据创建
        train_datas, train_labels = self.read_raw(DataDir+data_set[0])
        word2index, vocab_size = self.word_index(train_datas, vocab_size)
        test_datas, test_labels = self.read_raw(DataDir+data_set[1])
        
        train_features = []
        for data in train_datas:
            feature = []
            data_list = data.split()
            for word in data_list:
                word = word.lower() #词表中的单词均为小写
                if word in word2index:
                    feature.append(word2index[word])
                else:
                    feature.append(word2index["<unk>"]) #词表中未出现的词用<unk>代替
                if(len(feature)==max_len): #限制句子的最大长度，超出部分直接截断
                    break
            #对未达到最大长度的句子添加padding
            feature = feature + [word2index["<pad>"]] * (max_len - len(feature))
            train_features.append(feature)
            
        test_features = []
        for data in test_datas:
            feature = []
            data_list = data.split()
            for word in data_list:
                word = word.lower() #词表中的单词均为小写
                if word in word2index:
                    feature.append(word2index[word])
                else:
                    feature.append(word2index["<unk>"]) #词表中未出现的词用<unk>代替
                if(len(feature)==max_len): #限制句子的最大长度，超出部分直接截断
                    break
            #对未达到最大长度的句子添加padding
            feature = feature + [word2index["<pad>"]] * (max_len - len(feature))
            test_features.append(feature)
            
        #将词的index转换成tensor,train_features中数据的维度需要一致，否则会报错
        train_features = torch.LongTensor(train_features)
        train_labels = torch.FloatTensor(train_labels)
        
        test_features = torch.LongTensor(test_features)
        test_labels = torch.FloatTensor(test_labels)
        
        #将词转化为embedding
        #词表中有两个特殊的词<unk>和<pad>，所以词表实际大小为vocab_size + 2
        embed = nn.Embedding(vocab_size + 2, embedding_size)
        train_features = embed(train_features)
        test_features = embed(test_features)
        #指定输入特征是否需要计算梯度
        train_features = Variable(train_features, requires_grad=False)
        train_datasets = torch.utils.data.TensorDataset(train_features, train_labels)
        
        test_features = Variable(test_features, requires_grad=False)
        test_datasets = torch.utils.data.TensorDataset(test_features, test_labels)
        return train_datasets, test_datasets

    def get_Features(self,data_set,vocab_size, embedding_size, max_len):
        #注，由于nn.Embedding每次生成的词嵌入不固定，因此此处同时获取训练数据的词嵌入和测试数据的词嵌入
        #测试数据的词表也用训练数据创建
        train_datas, train_labels = self.read_raw(DataDir+data_set[0])
        word2index, vocab_size = self.word_index(train_datas, vocab_size)
        test_datas, test_labels = self.read_raw(DataDir+data_set[1])
        
        train_features = []
        for data in train_datas:
            feature = []
            data_list = data.split()
            for word in data_list:
                word = word.lower() #词表中的单词均为小写
                if word in word2index:
                    feature.append(word2index[word])
                else:
                    feature.append(word2index["<unk>"]) #词表中未出现的词用<unk>代替
                if(len(feature)==max_len): #限制句子的最大长度，超出部分直接截断
                    break
            #对未达到最大长度的句子添加padding
            feature = feature + [word2index["<pad>"]] * (max_len - len(feature))
            train_features.append(feature)
            
        test_features = []
        for data in test_datas:
            feature = []
            data_list = data.split()
            for word in data_list:
                word = word.lower() #词表中的单词均为小写
                if word in word2index:
                    feature.append(word2index[word])
                else:
                    feature.append(word2index["<unk>"]) #词表中未出现的词用<unk>代替
                if(len(feature)==max_len): #限制句子的最大长度，超出部分直接截断
                    break
            #对未达到最大长度的句子添加padding
            feature = feature + [word2index["<pad>"]] * (max_len - len(feature))
            test_features.append(feature)
            
        #将词的index转换成tensor,train_features中数据的维度需要一致，否则会报错
        train_features = torch.LongTensor(train_features)
        train_labels = torch.FloatTensor(train_labels)
        
        test_features = torch.LongTensor(test_features)
        test_labels = torch.FloatTensor(test_labels)
        
        #将词转化为embedding
        #词表中有两个特殊的词<unk>和<pad>，所以词表实际大小为vocab_size + 2
        embed = nn.Embedding(vocab_size + 2, embedding_size)
        train_features = embed(train_features)
        test_features = embed(test_features)
        #指定输入特征是否需要计算梯度
        train_features = Variable(train_features, requires_grad=False)
        
        test_features = Variable(test_features, requires_grad=False)
        return train_features, test_features


In [9]:
class BiLSTMModel(nn.Module):
    def __init__(self, embedding_size,hidden_size, num_layers, num_directions, num_classes):
        super(BiLSTMModel, self).__init__()
        
        self.input_size = embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_directions = num_directions
        
        
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers = num_layers, bidirectional = (num_directions == 2))
        self.attention_weights_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(inplace=True)
        )
        self.liner = nn.Linear(hidden_size, num_classes)
        self.act_func = nn.Softmax(dim=1)
    
    def forward(self, x):
        #lstm的输入维度为 [seq_len, batch, input_size]
        #x [batch_size, sentence_length, embedding_size]
        x = x.permute(1, 0, 2)         #[sentence_length, batch_size, embedding_size]
        #由于数据集不一定是预先设置的batch_size的整数倍，所以用size(1)获取当前数据实际的batch
        batch_size = x.size(1)
        
        #设置lstm最初的前项输出
        h_0 = torch.randn(self.num_layers * self.num_directions, batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_layers * self.num_directions, batch_size, self.hidden_size).to(device)
        
        #out[seq_len, batch, num_directions * hidden_size]。多层lstm，out只保存最后一层每个时间步t的输出h_t
        #h_n, c_n [num_layers * num_directions, batch, hidden_size]
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        
        #将双向lstm的输出拆分为前向输出和后向输出
        (forward_out, backward_out) = torch.chunk(out, 2, dim = 2)
        out = forward_out + backward_out  #[seq_len, batch, hidden_size]
        out = out.permute(1, 0, 2)  #[batch, seq_len, hidden_size]
        
        #为了使用到lstm最后一个时间步时，每层lstm的表达，用h_n生成attention的权重
        h_n = h_n.permute(1, 0, 2)  #[batch, num_layers * num_directions,  hidden_size]
        h_n = torch.sum(h_n, dim=1) #[batch, 1,  hidden_size]
        h_n = h_n.squeeze(dim=1)  #[batch, hidden_size]
        
        attention_w = self.attention_weights_layer(h_n)  #[batch, hidden_size]
        attention_w = attention_w.unsqueeze(dim=1) #[batch, 1, hidden_size]
        
        attention_context = torch.bmm(attention_w, out.transpose(1, 2))  #[batch, 1, seq_len]
        softmax_w = F.softmax(attention_context, dim=-1)  #[batch, 1, seq_len],权重归一化
        
        x = torch.bmm(softmax_w, out)  #[batch, 1, hidden_size]
        x = x.squeeze(dim=1)  #[batch, hidden_size]
        x = self.liner(x)
        x = self.act_func(x)
        return x
        
def test(model, test_loader, loss_func):
    model.eval()
    loss_val = 0.0
    corrects = 0.0
    for datas, labels in test_loader:
        datas = datas.to(device)
        labels = labels.to(device)
        
        preds = model(datas)
        loss = loss_func(preds, labels)
        
        loss_val += loss.item() * datas.size(0)
        
        #获取预测的最大概率出现的位置
        preds = torch.argmax(preds, dim=1)
        labels = torch.argmax(labels, dim=1)
        corrects += torch.sum(preds == labels).item()
    test_loss = loss_val / len(test_loader.dataset)
    test_acc = corrects / len(test_loader.dataset)
    print("Test Loss: {}, Test Acc: {}".format(test_loss, test_acc))
    return test_acc

def train(model, train_loader,test_loader, optimizer, loss_func, epochs):
    best_val_acc = 0.0
    best_model_params = copy.deepcopy(model.state_dict())
    for epoch in range(epochs):
        model.train()
        loss_val = 0.0
        corrects = 0.0
        for datas, labels in train_loader:
            datas = datas.to(device)
            labels = labels.to(device)
            
            preds = model(datas)
            loss = loss_func(preds, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_val += loss.item() * datas.size(0)
            
            #获取预测的最大概率出现的位置
            preds = torch.argmax(preds, dim=1)
            labels = torch.argmax(labels, dim=1)
            corrects += torch.sum(preds == labels).item()
        train_loss = loss_val / len(train_loader.dataset)
        train_acc = corrects / len(train_loader.dataset)
        if(epoch % 1 == 0):
            print(f"--------Epoch <<{epoch}>> Results:--------")
            print("Train Loss: {}, Train Acc: {}".format(train_loss, train_acc))
            test_acc = test(model, test_loader, loss_func)
            if(best_val_acc < test_acc):
                best_val_acc = test_acc
                best_model_params = copy.deepcopy(model.state_dict())
                print("Best model params updated!")
    model.load_state_dict(best_model_params)
    return model

In [7]:
Dload=DataLoader()
trSet,teSet=Dload.get_datasets(Set2Train,vocab_size=vocab_size,embedding_size=embedding_size,max_len=sentence_max_len)
train_loader = torch.utils.data.DataLoader(trSet, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(teSet, batch_size=batch_size, shuffle=True)

In [ ]:
model = BiLSTMModel(embedding_size, hidden_size, num_layers, num_directions, num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.BCELoss()
model = train(model, train_loader, test_loader, optimizer, loss_func, epochs)
torch.save(model,ModelDir)
print(f"Model saved to {ModelDir} !")

L4,Epoch10,Batch32

--------Epoch <<0>> Results:--------
Train Loss: 0.4866435846987773, Train Acc: 0.7935768616771528
Test Loss: 0.4605570047280944, Test Acc: 0.796378390562777
Best model params updated!
--------Epoch <<1>> Results:--------
Train Loss: 0.4676954279137123, Train Acc: 0.7946286036247535
Test Loss: 0.45619719377931944, Test Acc: 0.796378390562777
--------Epoch <<2>> Results:--------
Train Loss: 0.46273632308039514, Train Acc: 0.7946286036247535
Test Loss: 0.4555758065326091, Test Acc: 0.796378390562777
--------Epoch <<3>> Results:--------
Train Loss: 0.45822107717569743, Train Acc: 0.794572260306132
Test Loss: 0.4521541875388981, Test Acc: 0.7964535276880307
Best model params updated!
--------Epoch <<4>> Results:--------
Train Loss: 0.4535511062966679, Train Acc: 0.7969011174758194
Test Loss: 0.4536914214277995, Test Acc: 0.798933052821399
Best model params updated!
--------Epoch <<5>> Results:--------
Train Loss: 0.45050139221767604, Train Acc: 0.7978589538923843
Test Loss: 0.45004856531370485, Test Acc: 0.7992336013224134
Best model params updated!
--------Epoch <<6>> Results:--------
Train Loss: 0.4475180268130937, Train Acc: 0.7998309700441356
Test Loss: 0.4452496790992962, Test Acc: 0.7998346983244421
Best model params updated!
--------Epoch <<7>> Results:--------
Train Loss: 0.44521234057544934, Train Acc: 0.8006385576110433
Test Loss: 0.4503334618405653, Test Acc: 0.7983319558193703
--------Epoch <<8>> Results:--------
Train Loss: 0.4413476846130569, Train Acc: 0.8033054746924594
Test Loss: 0.44496255158007625, Test Acc: 0.7979562701931024
--------Epoch <<9>> Results:--------
Train Loss: 0.4388651643675676, Train Acc: 0.80409428115316
Test Loss: 0.44902093252265773, Test Acc: 0.7993838755729206
Model saved to Models/bi-LSTM-all_l4_e10.pth !


In [45]:
from sklearn.metrics import confusion_matrix,matthews_corrcoef,classification_report

def GetPrediction(features,model_path,batch):
    model = torch.load(model_path)
    length=len(features)
    c_ind=0
    out=torch.Tensor()
    out=out.cuda(0)
    while(c_ind<length):
        if length-c_ind>batch:
            l=batch
        else:
            l=1
        pred=model(features[c_ind:c_ind+l].cuda(0))
        out=torch.cat((out,pred))
        c_ind=c_ind+l
    return out

def EvaluatePair(Dset,result):
    dfte=pd.read_csv(DataDir+"/"+Dset[1])
    dfpred=pd.DataFrame(result,columns=["0","1"])
    dfpred["Pred"]=dfpred["0"]<=dfpred["1"]
    dfpred["Pred"]=dfpred["Pred"].apply(lambda x:0 if x==True else 1)
    cm=confusion_matrix(dfte["y"],dfpred["Pred"])
    mc=matthews_corrcoef(dfte["y"],dfpred["Pred"])
    target_names = ['No crystal', 'Crystal']
    print("-------%s Metrics on test set are: " % Dset[1])
    print("----Confusion Matrix:")
    print(cm)
    print("----Matthews Corrcoef:")
    print(mc)
    print(classification_report(dfte["y"], dfpred["Pred"], target_names=target_names))


In [46]:
for d in DataPairs[0:4]:
    Dload1=DataLoader()
    trFeatures,teFeatures=Dload1.get_Features(d,vocab_size=vocab_size,embedding_size=embedding_size,max_len=sentence_max_len)
    result=GetPrediction(teFeatures,"Models/bi-LSTM-all_l4_e10.pth",100)
    EvaluatePair(d,result)

-------CF_TE_Sequence.fasta.csv Metrics on test set are: 
----Confusion Matrix:
[[ 66  77]
 [174 229]]
----Matthews Corrcoef:
0.026377304180641305
              precision    recall  f1-score   support

  No crystal       0.28      0.46      0.34       143
     Crystal       0.75      0.57      0.65       403

    accuracy                           0.54       546
   macro avg       0.51      0.51      0.50       546
weighted avg       0.62      0.54      0.57       546

-------CRYS_TE_Sequence.fasta.csv Metrics on test set are: 
----Confusion Matrix:
[[3647  979]
 [ 276   45]]
----Matthews Corrcoef:
-0.04343660694468044
              precision    recall  f1-score   support

  No crystal       0.93      0.79      0.85      4626
     Crystal       0.04      0.14      0.07       321

    accuracy                           0.75      4947
   macro avg       0.49      0.46      0.46      4947
weighted avg       0.87      0.75      0.80      4947

-------MC_TE_Sequence.fasta.csv Metrics on tes

C:\ProgramData\Anaconda3\envs\nlp38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\nlp38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\nlp38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-------MF_TE_Sequence.fasta.csv Metrics on test set are: 
----Confusion Matrix:
[[ 456 3092]
 [ 201 1198]]
----Matthews Corrcoef:
-0.020106151333621612
              precision    recall  f1-score   support

  No crystal       0.69      0.13      0.22      3548
     Crystal       0.28      0.86      0.42      1399

    accuracy                           0.33      4947
   macro avg       0.49      0.49      0.32      4947
weighted avg       0.58      0.33      0.27      4947



-------CF_TE_Sequence.fasta.csv Metrics on test set are: 
----Confusion Matrix:
[[ 66  77]
 [174 229]]
----Matthews Corrcoef:
0.026377304180641305
              precision    recall  f1-score   support

  No crystal       0.28      0.46      0.34       143
     Crystal       0.75      0.57      0.65       403

    accuracy                           0.54       546
   macro avg       0.51      0.51      0.50       546
weighted avg       0.62      0.54      0.57       546

-------CRYS_TE_Sequence.fasta.csv Metrics on test set are: 
----Confusion Matrix:
[[3647  979]
 [ 276   45]]
----Matthews Corrcoef:
-0.04343660694468044
              precision    recall  f1-score   support

  No crystal       0.93      0.79      0.85      4626
     Crystal       0.04      0.14      0.07       321

    accuracy                           0.75      4947
   macro avg       0.49      0.46      0.46      4947
weighted avg       0.87      0.75      0.80      4947

-------MC_TE_Sequence.fasta.csv Metrics on test set are: 
----Confusion Matrix:
[[891   0]
 [129   0]]
----Matthews Corrcoef:
0.0
              precision    recall  f1-score   support

  No crystal       0.87      1.00      0.93       891
     Crystal       0.00      0.00      0.00       129

    accuracy                           0.87      1020
   macro avg       0.44      0.50      0.47      1020
weighted avg       0.76      0.87      0.81      1020
